In [82]:
import json, requests as rq, datetime as dt
import pandas as pd

class DFCols:
    REGION = 'Регіон'
    FACILITY = 'Заклад'
    EDRPOU = 'код ЄДРПОУ'
    VACCINE = 'Міжнародна непатентована назва'
    VACCINE_SERIES = 'Серія вакцини'
    EXPIRATION_DATE = 'Термін придатності'
    AMOUNT = 'Кількість доз'
    ACCOUNTING_DATE = 'Дата обліку'
    REQUIRED_COLUMNS = [EDRPOU, FACILITY, VACCINE_SERIES, VACCINE, EXPIRATION_DATE, AMOUNT, ACCOUNTING_DATE, REGION]

# ––––––––––––––––––––––––––––––––––––––––– Download facility-level stock data –––––––––––––––––––––––––––––––––––––––––
# df = pd.DataFrame()
# for region in ['Вінницька', 'Волинська', 'Дніпропетровська', 'Донецька',
#        'Житомирська', 'Закарпатська', 'Івано-Франківська', 'Київська',
#        'Кіровоградська', 'м. Київ', 'Львівська', 'Миколаївська',
#        'Одеська', 'Сумська', 'Полтавська', 'Рівненська', 'Тернопільська',
#        'Харківська', 'Херсонська', 'Хмельницька', 'Черкаська',
#        'Чернігівська', 'Чернівецька', 'Запорізька', 'Україна']:
#     region = region.replace('м. ', '').upper()
#     print(region, end='. ')
#     try:
#         rp = rq.get(f'https://vaccine.meddata.com.ua/index.php?\
#             option=com_fabrik&task=meddata.safemed&method=getList&format=raw&region={region}')
#         df = pd.concat([df, pd.DataFrame(rp.json()['data'])], ignore_index=True, axis=0)
#         print('Successfully processed.')
#     except json.JSONDecodeError as e:
#         print(f'Error has happened. {e}')
# df.to_csv('Results/MedData.csv', index=False, sep=';')
# –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

# –––––––––––––––––––––––––––––––––––––––––––– Download regional stock data ––––––––––––––––––––––––––––––––––––––––––––
rp = rq.get('https://vaccine.meddata.com.ua/index.php?\
    option=com_fabrik&task=meddata.safemed&\
        method=vaccines_regional&format=raw')
regional_stock = pd.DataFrame(rp.json()['data'])
regional_stock.to_csv('Results/RegionalStock.csv', index=False, sep=';')

regional_stock = pd.read_csv('Results/RegionalStock.csv', sep=';')
regional_stock.drop(columns=['manufacture_date'], inplace=True)
regional_stock.rename(
    columns={
        'zaklad': 'name',
        'vaccine_name': 'trade_name',
        'updated': 'accounting_date',
        'defrosting_date': 'expire_date_defrosting',
        }, 
    inplace=True)

regional_stock.loc[regional_stock['edrpou']==38502841, 'name'] = 'ДЕРЖАВНА УСТАНОВА "ПОЛТАВСЬКИЙ ОБЛАСНИЙ ЦЕНТР КОНТРОЛЮ ТА ПРОФІЛАКТИКИ ХВОРОБ МІНІСТЕРСТВА ОХОРОНИ ЗДОРОВ\'Я УКРАЇНИ"'
regional_stock['accounting_date'] = pd.to_datetime(regional_stock['accounting_date'])
regional_stock['expire_date'] = pd.to_datetime(regional_stock['expire_date'])
regional_stock['expire_date_defrosting'] = pd.to_datetime(regional_stock['expire_date_defrosting'])
regional_stock['region_order'] = regional_stock['name'].str.extract(r'(?i)(\w+-?\w+(?:с|ц|з)ький)')[0].str.replace('(?i)ий$', 'а', regex=True).str.title()
# –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––


# –––––––––––––––––––––––––––––––––––––––––––––––– Data processing begins –––––––––––––––––––––––––––––––––––––––––––––––
df = pd.read_csv('Results/MedData.csv', sep=';')

df['region_order'] = df['region_order'].str.title()
df['edrpou'] = df['edrpou'].astype(str)
df['accounting_date'] = pd.to_datetime(df['accounting_date'])
df['expire_date'] = pd.to_datetime(df['expire_date'])
df['expire_date_defrosting'] = pd.to_datetime(df['expire_date_defrosting'])

df.dropna(subset=['accounting_date'], inplace=True) # Potential loss of data

df = pd.concat([df, regional_stock], ignore_index=True, axis=0) # Add regional stock data to facility-level data

# Replace expire_date with expire_date_defrosting if the latter exists and is less than the former
expire_date_replacement = (df['expire_date_defrosting'].notna()) & (df['expire_date_defrosting'] < df['expire_date'])
df.loc[ 
    expire_date_replacement, 
    'expire_date'
] = df.loc[expire_date_replacement, 'expire_date_defrosting']
df = df.drop(columns=['expire_date_defrosting'])

# Compute the amount of consumed vaccines for the last "pure" 6 months
month_period = 6
now = dt.datetime.now()
start_date = dt.datetime(
    now.year + (now.month - month_period) // 12, 
    (now.month - 6) % 12 + 1,
    1
)
end_date = dt.datetime(now.year, now.month, 1) - dt.timedelta(days=1)

usage = df[
    df['series'].isin(['GA2988', 'GK0925', 'HJ3099', 'GC9425', 'HL8354']) # Specific series
    &
    (df['accounting_date'].between(start_date, end_date)) # Last 6 months
].groupby(
    [df['accounting_date'].dt.to_period('M'), 'region_order']
)[['cons_quantity', 'broken']].sum()


df = df.groupby([
    'region_order', 
    'name', 
    'edrpou', 
    'trade_name', 
    'series', 
    'expire_date'
]).apply(lambda xs: xs[xs['accounting_date'] == xs['accounting_date'].max()][[
    'cons_quantity', 
    'broken', 
    'balance_quantity', 
    'accounting_date',
]].iloc[0], include_groups=False)

df.reset_index(inplace=True)
df.rename(columns={
    'region_order': DFCols.REGION,
    'name': DFCols.FACILITY, 
    'edrpou': DFCols.EDRPOU, 
    'trade_name': DFCols.VACCINE, 
    'series': DFCols.VACCINE_SERIES,
    'expire_date': DFCols.EXPIRATION_DATE, 
    'balance_quantity': DFCols.AMOUNT, 
    'accounting_date': DFCols.ACCOUNTING_DATE}, inplace=True)
df = df[DFCols.REQUIRED_COLUMNS]
# df = df.drop(columns=['Unnamed: 0'])
df

/var/folders/gc/9t_hywd97l7fb1yjstm49l1w0000gn/T/ipykernel_93494/1866036.py:62: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Results/MedData.csv', sep=';')


,код ЄДРПОУ,Заклад,Серія вакцини,Міжнародна непатентована назва,Термін придатності,Кількість доз,Дата обліку,Регіон
0,8734204,"ДУ ""ТМО МВС України по Івано-Франківській обла...",GD6799,COMIRNATY™ Original/Omicron BA.1,2023-04-30 00:00:00,0,2022-11-30,Івано-Франківська
1,1993256,"КНП ""ІФ ОДКЛ ІФ ОР""",GK0925,"Pfizer-BioNTech COVID-19 Vaccine, Bivalent Ori...",2024-02-12 00:00:00,0,2023-12-28,Івано-Франківська
2,42244782,"КНП ""БУРШТИНСЬКИЙ МІСЬКИЙ ЦЕНТР ПЕРВИННОЇ МЕДИ...",GC9425,COMIRNATYTM Tris/Sucrose,2023-11-14 00:00:00,0,2023-11-02,Івано-Франківська
3,42244782,"КНП ""БУРШТИНСЬКИЙ МІСЬКИЙ ЦЕНТР ПЕРВИННОЇ МЕДИ...",GC9425,COMIRNATYTM Tris/Sucrose,2024-03-30 00:00:00,0,2023-03-01,Івано-Франківська
4,42244782,"КНП ""БУРШТИНСЬКИЙ МІСЬКИЙ ЦЕНТР ПЕРВИННОЇ МЕДИ...",GD6799,COMIRNATY™ Original/Omicron BA.1,2023-04-30 00:00:00,0,2022-12-13,Івано-Франківська
...,...,...,...,...,...,...,...,...
13039,3315913109,ФОП Сердюк Марія Юріївна,GC9425,COMIRNATYTM Tris/Sucrose,2023-11-17 00:00:00,0,2023-10-25,Чернігівська
13040,3315913109,ФОП Сердюк Марія Юріївна,GC9425,COMIRNATYTM Tris/Sucrose,2024-03-13 00:00:00,0,2024-02-25,Чернігівська
13041,3315913109,ФОП Сердюк Марія Юріївна,GK0925,"Pfizer-BioNTech COVID-19 Vaccine, Bivalent Ori...",2024-05-15 00:00:00,0,2024-05-14,Чернігівська
13042,38509742,"ЦКПХ ДУ ""Чернігівський ОЦКПХ МОЗ України""",HL8354,COMIRNATY Omicron XBB.1.5 (child),2024-03-04 10:48:21,0,2024-04-03,Чернігівська


In [28]:
# df = pd.read_csv('Results/MedData.csv', sep=';')
df

,region_order,name,edrpou,trade_name,series,expire_date,expire_date_defrosting,cons_quantity,broken,balance_quantity,accounting_date
0,ВІННИЦЬКА,"КНП ""Вінницька обласна клінічна психоневрологі...",1982264,COMIRNATY™ Original/Omicron BA.1,GD6799,2023-04-30,NaN,0,0.0,0,2022-10-13
1,ВІННИЦЬКА,"КНП ""Вінницька обласна клінічна психоневрологі...",1982264,COMIRNATY™ Original/Omicron BA.1,GD6799,2023-04-30,NaN,0,0.0,0,2022-10-16
2,ВІННИЦЬКА,"КНП ""Вінницька обласна клінічна психоневрологі...",1982264,COMIRNATY™ Original/Omicron BA.1,GD6799,2023-04-30,NaN,0,0.0,0,2022-10-17
3,ВІННИЦЬКА,"КНП ""Вінницька обласна клінічна психоневрологі...",1982264,COMIRNATY™ Original/Omicron BA.1,GD6799,2023-04-30,NaN,0,0.0,0,2022-10-18
4,ВІННИЦЬКА,"КНП ""Вінницька обласна клінічна психоневрологі...",1982264,COMIRNATY™ Original/Omicron BA.1,GD6799,2023-04-30,NaN,0,0.0,0,2022-10-19
...,...,...,...,...,...,...,...,...,...,...,...
1149562,Запорізька,САС ЮЛІЯ КОСТЯНТИНІВНА,sasfop2019@gmail.com,"Pfizer-BioNTech COVID-19 Vaccine, Bivalent Ori...",GK0925,2024-08-31,2024-03-14,0,0.0,0,2024-04-14
1149563,Запорізька,САС ЮЛІЯ КОСТЯНТИНІВНА,sasfop2019@gmail.com,"Pfizer-BioNTech COVID-19 Vaccine, Bivalent Ori...",GK0925,2024-08-31,2024-03-14,0,0.0,0,2024-04-13
1149564,Запорізька,САС ЮЛІЯ КОСТЯНТИНІВНА,sasfop2019@gmail.com,"Pfizer-BioNTech COVID-19 Vaccine, Bivalent Ori...",GK0925,2024-08-31,2024-03-14,0,0.0,0,2024-04-12
1149565,Запорізька,САС ЮЛІЯ КОСТЯНТИНІВНА,sasfop2019@gmail.com,"Pfizer-BioNTech COVID-19 Vaccine, Bivalent Ori...",GK0925,2024-08-31,2024-03-14,0,0.0,0,2024-04-11


In [1]:
from src import get_national_leftovers, get_data, _get_usage_meddata, get_usage
get_data(refresh=True)
d = _get_usage_meddata()
d

––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
2024-05-26 22:30:49,706 :: INFO
Starting to process MedData
––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
2024-05-26 22:30:49,707 :: INFO
Reading national leftovers from MedData
––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
2024-05-26 22:30:49,899 :: INFO
Picking existing MedData data.
––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
2024-05-26 22:31:05,212 :: INFO
Data loaded. 2024-05-25 is the file's report date.
––––––––––––

,Дата,Регіон,Міжнародна непатентована назва,Кількість зроблених у звітному місяці щеплень,Використано доз за звітний місяць при проведенні щеплень
0,2023-12-01,Івано-Франківська,Pfizer,355.0,468.0
1,2023-12-01,Волинська,Pfizer,122.0,491.0
2,2023-12-01,Вінницька,Pfizer,482.0,747.0
3,2023-12-01,Дніпропетровська,Pfizer,3782.0,4224.0
4,2023-12-01,Донецька,Pfizer,123.0,163.0
...,...,...,...,...,...
124,2024-04-01,Черкаська,Pfizer,55.0,157.0
125,2024-04-01,Чернівецька,Pfizer,204.0,270.0
126,2024-04-01,Чернівецька,Pfizer (дитяча),0.0,0.0
127,2024-04-01,Чернігівська,Pfizer,141.0,271.0


In [1]:
import pandas as pd
from src import get_data
df, REPORT_DATE, timed_outs, national_leftovers = get_data(refresh=True)

––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
2024-05-26 23:08:17,950 :: INFO
Starting to process MedData
––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
2024-05-26 23:08:17,950 :: INFO
Reading national leftovers from MedData
––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
2024-05-26 23:08:18,307 :: INFO
Picking existing MedData data.
––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
2024-05-26 23:08:27,998 :: INFO
Data loaded. 2024-05-25 is the file's report date.
––––––––––––

In [9]:
timed_outs

,код ЄДРПОУ,Заклад,Серія препарату,Міжнародна непатентована назва,Термін придатності,Кількість доз,дата оновлення,Регіон,Джерело фінансування
0,8734204,"ДУ ""ТМО МВС України по Івано-Франківській обла...",GD6799,Pfizer,2023-04-30 00:00:00,0,2022-11-30,Івано-Франківська,Covax
1,1993256,"КНП ""ІФ ОДКЛ ІФ ОР""",GK0925,Pfizer,2024-02-12 00:00:00,0,2023-12-28,Івано-Франківська,Covax
2,42244782,"КНП ""БУРШТИНСЬКИЙ МІСЬКИЙ ЦЕНТР ПЕРВИННОЇ МЕДИ...",XE769,Jcovden,2023-11-30 00:00:00,0,2022-09-16,Івано-Франківська,Covax
3,42244782,"КНП ""БУРШТИНСЬКИЙ МІСЬКИЙ ЦЕНТР ПЕРВИННОЇ МЕДИ...",XE770,Jcovden,2023-11-30 00:00:00,0,2023-04-27,Івано-Франківська,Covax
4,42244782,"КНП ""БУРШТИНСЬКИЙ МІСЬКИЙ ЦЕНТР ПЕРВИННОЇ МЕДИ...",FY1793,Pfizer,2022-11-30 00:00:00,0,2022-11-25,Івано-Франківська,Covax
...,...,...,...,...,...,...,...,...,...
13129,3315913109,ФОП Сердюк Марія Юріївна,GC9425,Pfizer,2023-11-17 00:00:00,0,2023-10-25,Чернігівська,Covax
13130,3315913109,ФОП Сердюк Марія Юріївна,GC9425,Pfizer,2024-03-13 00:00:00,0,2024-02-25,Чернігівська,Covax
13131,3315913109,ФОП Сердюк Марія Юріївна,GK0925,Pfizer,2024-05-15 00:00:00,0,2024-05-14,Чернігівська,Covax
13132,38509742,"ЦКПХ ДУ ""Чернігівський ОЦКПХ МОЗ України""",GK0925,Pfizer,2023-11-29 19:51:05,0,2023-12-04,Чернігівська,Covax


In [16]:
import pandas as pd, requests as rq
rp = rq.get('https://vaccine.meddata.com.ua/index.php?\
    option=com_fabrik&task=meddata.safemed&\
         method=vaccines_regional&format=raw')

regional_stock = pd.DataFrame(rp.json()['data'])

regional_stock.drop(columns=['manufacture_date'], inplace=True)
regional_stock.rename(
    columns={
        'zaklad': 'name',
        'vaccine_name': 'trade_name',
        'updated': 'accounting_date',
        'defrosting_date': 'expire_date_defrosting',
        }, 
    inplace=True)
regional_stock.loc[regional_stock['edrpou']==38502841, 'name'] = 'ДЕРЖАВНА УСТАНОВА "ПОЛТАВСЬКИЙ ОБЛАСНИЙ ЦЕНТР КОНТРОЛЮ ТА ПРОФІЛАКТИКИ ХВОРОБ МІНІСТЕРСТВА ОХОРОНИ ЗДОРОВ\'Я УКРАЇНИ"'
regional_stock['accounting_date'] = pd.to_datetime(regional_stock['accounting_date'])
regional_stock['expire_date'] = pd.to_datetime(regional_stock['expire_date'])
regional_stock['expire_date_defrosting'] = pd.to_datetime(regional_stock['expire_date_defrosting'])
regional_stock['region_order'] = regional_stock['name'].str.extract(r'(?i)(\w+-?\w+(?:с|ц|з)ький)')[0].str.replace('(?i)ий$', 'а', regex=True).str.title()
regional_stock

,edrpou,name,trade_name,series,balance_quantity,expire_date,expire_date_defrosting,accounting_date,region_order
0,38331800,"ЦКПХ ДУ ""Івано-Франківський ОЦКПХ МОЗ України""",COMIRNATY Omicron XBB.1.5 (child),HL8354,0,2025-03-31,2024-03-04 10:48:21,2024-04-03,Івано-Франківська
1,38331800,"ЦКПХ ДУ ""Івано-Франківський ОЦКПХ МОЗ України""",PFE-BNT 0.1mg/ml 10x2.25ml MDV WT-BA4/5 (Pfize...,GK0925,0,2024-08-31,2023-11-29 19:51:05,2023-12-04,Івано-Франківська
2,38512294,ЦКПХ ДУ «Вінницький ОЦКПХ МОЗ України»,COMIRNATY Omicron XBB.1.5 (child),HL8354,70,2025-03-31,2024-03-04 10:48:21,2024-04-03,Вінницька
3,38512294,ЦКПХ ДУ «Вінницький ОЦКПХ МОЗ України»,PFE-BNT 0.1mg/ml 10x2.25ml MDV WT-BA4/5 (Pfize...,GK0925,480,2024-08-31,2023-11-29 19:51:05,2023-12-04,Вінницька
4,38474592,"ЦКПХ ДУ ""Волинський ОЦКПХ МОЗ України""",COMIRNATY Omicron XBB.1.5 (child),HL8354,10,2025-03-31,2024-03-04 10:48:21,2024-04-03,Волинська
5,38474592,"ЦКПХ ДУ ""Волинський ОЦКПХ МОЗ України""",PFE-BNT 0.1mg/ml 10x2.25ml MDV WT-BA4/5 (Pfize...,GK0925,138,2024-08-31,2023-11-29 19:51:05,2023-12-04,Волинська
6,38431598,"ЦКПХ ДУ ""Дніпропетровський ОЦКПХ МОЗ України""",COMIRNATY Omicron XBB.1.5 (child),HL8354,100,2025-03-31,2024-03-04 10:48:21,2024-04-03,Дніпропетровська
7,38431598,"ЦКПХ ДУ ""Дніпропетровський ОЦКПХ МОЗ України""",PFE-BNT 0.1mg/ml 10x2.25ml MDV WT-BA4/5 (Pfize...,GK0925,594,2024-08-31,2023-11-29 19:51:05,2023-12-04,Дніпропетровська
8,38531102,"ЦКПХ ДУ ""Донецький ОЦКПХ МОЗ України""",PFE-BNT 0.1mg/ml 10x2.25ml MDV WT-BA4/5 (Pfize...,GK0925,0,2024-08-31,2023-11-29 19:51:05,2024-03-04,Донецька
9,38499986,ЦКПХ ДУ «Житомирський ОЦКПХ МОЗ України»,COMIRNATY Omicron XBB.1.5 (child),HL8354,0,2025-03-31,2024-03-04 10:48:21,2024-05-15,Житомирська


In [26]:
import  datetime
regional_stock[
    # (regional_stock['expire_date_defrosting'] > datetime.datetime.today())
    # &
    (regional_stock['series'] == 'HL8354')
    &
    (regional_stock['region_order'] == 'Вінницька')]

datetime.datetime.fromisoformat('2024-03-04') + datetime.timedelta(days=7*10)

datetime.datetime(2024, 5, 13, 0, 0)

In [78]:

# df = pd.read_csv('Results/MedData.csv', sep=';')

df['region_order'] = df['region_order'].str.title()
df['edrpou'] = df['edrpou'].astype(str)
df['accounting_date'] = pd.to_datetime(df['accounting_date'])
df['expire_date'] = pd.to_datetime(df['expire_date'])

df.dropna(subset=['accounting_date'], inplace=True) # Potential loss of data


,region_order,name,edrpou,trade_name,series,expire_date,expire_date_defrosting,cons_quantity,broken,balance_quantity,accounting_date
0,Вінницька,"КНП ""Вінницька обласна клінічна психоневрологі...",1982264,COMIRNATY™ Original/Omicron BA.1,GD6799,2023-04-30,NaN,0.0,0.0,0,2022-10-13
1,Вінницька,"КНП ""Вінницька обласна клінічна психоневрологі...",1982264,COMIRNATY™ Original/Omicron BA.1,GD6799,2023-04-30,NaN,0.0,0.0,0,2022-10-16
2,Вінницька,"КНП ""Вінницька обласна клінічна психоневрологі...",1982264,COMIRNATY™ Original/Omicron BA.1,GD6799,2023-04-30,NaN,0.0,0.0,0,2022-10-17
3,Вінницька,"КНП ""Вінницька обласна клінічна психоневрологі...",1982264,COMIRNATY™ Original/Omicron BA.1,GD6799,2023-04-30,NaN,0.0,0.0,0,2022-10-18
4,Вінницька,"КНП ""Вінницька обласна клінічна психоневрологі...",1982264,COMIRNATY™ Original/Omicron BA.1,GD6799,2023-04-30,NaN,0.0,0.0,0,2022-10-19
...,...,...,...,...,...,...,...,...,...,...,...
1146765,Черкаська,"ЦКПХ ДУ ""Черкаський ОЦКПХ МОЗ України""",38469768,PFE-BNT 0.1mg/ml 10x2.25ml MDV WT-BA4/5 (Pfize...,GK0925,2024-08-31,2023-11-29 19:51:05,NaN,NaN,24,2023-12-04
1146766,Чернівецька,"ЦКПХ ДУ ""Чернівецький ОЦКПХ МОЗ України""",38453009,COMIRNATY Omicron XBB.1.5 (child),HL8354,2025-03-31,2024-03-04 10:48:21,NaN,NaN,110,2024-04-03
1146767,Чернівецька,"ЦКПХ ДУ ""Чернівецький ОЦКПХ МОЗ України""",38453009,PFE-BNT 0.1mg/ml 10x2.25ml MDV WT-BA4/5 (Pfize...,GK0925,2024-08-31,2023-11-29 19:51:05,NaN,NaN,414,2024-03-04
1146768,Чернігівська,"ЦКПХ ДУ ""Чернігівський ОЦКПХ МОЗ України""",38509742,COMIRNATY Omicron XBB.1.5 (child),HL8354,2025-03-31,2024-03-04 10:48:21,NaN,NaN,0,2024-04-03


In [14]:
from src import get_national_leftovers, _get_national_stock_meddata, get_data
import pandas as pd, requests as rq
# rp = rq.get('https://vaccine.meddata.com.ua/index.php?\
#     option=com_fabrik&task=meddata.safemed&\
#         method=vaccines_national&format=raw')
# nat_stock = pd.DataFrame(rp.json()['data']) 
# nat_stock
# df, *_ = get_data()
df[df['Міжнародна непатентована назва'] == "Pfizer (дитячий)"]

,код ЄДРПОУ,Заклад,Серія препарату,Міжнародна непатентована назва,Термін придатності,Кількість доз,дата оновлення,Регіон,Джерело фінансування
479,NaN,NaN,NaN,Pfizer (дитячий),2025-03-31,49900,NaT,Україна,Covax


,Вакцина,Регіон,Термін придатності,Залишок,Коментар,Джерело фінансування,Дата поставки,Відповідальний за імпорт
0,Pfizer,Нац Рівень,2024-08-31,106680,ФС,Covax,<NA>,<NA>
1,Pfizer,Нац Рівень,2025-03-31,201600,ФС,Covax,<NA>,<NA>
2,Pfizer (дитячий),Нац Рівень,2025-03-31,49900,ФС,Covax,<NA>,<NA>
